In [1]:
!pip install psycopg2-binary
!pip install yfinance


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install torch

  Using cached torch-2.5.1-cp312-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-75.5.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.5.1-cp312-none-macosx_11_0_arm64.whl (63.9 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached setuptools-75.5.0-py3-none-any.whl (1.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[

In [24]:
!pip install scikit-learn

  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl (23.1 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [54]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.functional as F
import psycopg2
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from torch.utils.data import TensorDataset, DataLoader

In [11]:
def stock_ingestion(ticker_list):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1825)
    
    stock_data = {}
    
    for ticker in ticker_list:
        try:
            stock = yf.Ticker(ticker)
            hist_data = stock.history(
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d'),
                interval='1d'
            )
            hist_data['Ticker'] = ticker
            
            stock_data[ticker] = hist_data
        except Exception as e:
            print(f'could not rertieve info for {ticker}: {str(e)}')
    
    combined_data = pd.concat(stock_data.values())
    combined_data.reset_index(inplace=True)
    combined_data = combined_data[['Ticker', 'Date'] + [col for col in combined_data.columns if col not in ['Ticker', 'Date']]]
    
    
    return combined_data
    
ticker_list = ['MSFT', 'AAPL', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'JPM', 'V', 'MA', 'BAC', 
               'GS', 'JNJ', 'UNH', 'PFE', 'ABBV', 'WMT', 'PG','KO', 'PEP', 'CAT', 'BA', 'HON', 'DIS', 
               'NFLX', 'INTC', 'AMD', 'QCOM', 'XOM', 'CVX']
stock_data = stock_ingestion(ticker_list)

In [13]:
stock_data.info

<bound method DataFrame.info of       Ticker                      Date        Open        High         Low  \
0       MSFT 2019-11-19 00:00:00-05:00  144.112987  144.542802  143.463478   
1       MSFT 2019-11-20 00:00:00-05:00  144.057096  144.565047  142.284065   
2       MSFT 2019-11-21 00:00:00-05:00  143.184969  143.568338  142.322414   
3       MSFT 2019-11-22 00:00:00-05:00  143.827093  144.047521  142.629093   
4       MSFT 2019-11-25 00:00:00-05:00  143.760015  145.053861  143.683341   
...      ...                       ...         ...         ...         ...   
37705    CVX 2024-11-11 00:00:00-05:00  157.139999  157.940002  155.899994   
37706    CVX 2024-11-12 00:00:00-05:00  157.300003  157.470001  155.119995   
37707    CVX 2024-11-13 00:00:00-05:00  156.070007  159.020004  154.529999   
37708    CVX 2024-11-14 00:00:00-05:00  159.839996  161.830002  159.679993   
37709    CVX 2024-11-15 00:00:00-05:00  161.520004  162.380005  160.759995   

            Close    Volume  Di

In [14]:
stock_data.shape

(37710, 9)

In [15]:
stock_data.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,37710.000000,37710.000000,37710.000000,37710.000000,3.771000e+04,37710.000000,37710.000000
mean,168.698106,170.687184,166.687501,168.720364,3.936483e+07,0.009442,0.001858
std,124.412268,125.785261,123.017337,124.427389,9.082301e+07,0.104442,0.160879
min,4.984939,5.174166,4.501143,4.892763,4.601000e+05,0.000000,0.000000
25%,79.991919,80.953674,78.862120,79.878531,5.450325e+06,0.000000,0.000000
50%,144.849906,146.307168,143.279384,144.765503,1.238540e+07,0.000000,0.000000
75%,208.986257,211.448502,206.471741,209.126301,3.390808e+07,0.000000,0.000000
max,833.669983,841.000000,830.020020,837.260010,1.543911e+09,3.000000,20.000000


In [16]:
print(len(stock_data))

37710


In [17]:
stock_data.head()

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,MSFT,2019-11-19 00:00:00-05:00,144.112987,144.542802,143.463478,143.644958,23935700,0.00,0.0
1,MSFT,2019-11-20 00:00:00-05:00,144.057096,144.565047,142.284065,143.395798,25696800,0.51,0.0
2,MSFT,2019-11-21 00:00:00-05:00,143.184969,143.568338,142.322414,143.261642,18576100,0.00,0.0
3,MSFT,2019-11-22 00:00:00-05:00,143.827093,144.047521,142.629093,143.367050,15901800,0.00,0.0
4,MSFT,2019-11-25 00:00:00-05:00,143.760015,145.053861,143.683341,144.938843,22420900,0.00,0.0


In [69]:
def temporal_preprocessing(df, sequence_length=10, test_size=0.3):
    df['Date'] = pd.to_datetime(df['Date'])
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day 
    df['day_of_week'] = df['Date'].dt.dayofweek 
    
    features = ['Open', 'High', 'Low', 'Close', 'Volume', 
               'year', 'month', 'day', 'day_of_week']
    data = df[features].values
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    
    X, y = [], []
    
    for i in range(len(data_scaled) - sequence_length):
        X.append(data_scaled[i:(i + sequence_length)])
        y.append(data_scaled[i + sequence_length, 3]) #3 = index for close price (dependent variable)
    
    X, y = np.array(X), np.array(y)
    
    train_size = int(len(X) * (1 - test_size))
    
    X_train = X[:train_size]
    X_test = X[train_size:]
    y_train = y[:train_size]
    y_test = y[train_size:]
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
    
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = DataLoader(dataset=dataset, batch_size=32, 
                            shuffle=True, pin_memory=torch.cuda.is_available)

    return X_test, y_test, dataloader

X_test, y_test, dataloader = temporal_preprocessing(stock_data)
X_test.shape

(11310, 10, 9)

In [70]:
input_size = 9 #num features 
hidden_size = 64
num_layers = 2
output_size = 1

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, 
                            hidden_size=hidden_size, 
                            layers=num_layers,
                            batch_first=True, 
                            dropout=0.2
                           )
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward():
        pass 
    